In [1]:
from utils.tf import *
from utils.db import *
from utils.gd2 import *
from utils.clt_repetidos import *
from datetime import datetime, time, timedelta
import cx_Oracle
import pandas as pd
import os
from utils.clt_repetidos import * 
from datetime import datetime
import re
import polars as pl
import gc
import polars as pl
from rapidfuzz import fuzz
from multiprocessing import Pool, cpu_count

# CLIENTES TAS

In [2]:
df_tas = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/test/data/extraer/30_05_2025/Clientes_activos_TAS.dat")

In [68]:
df_tas[df_tas["NOMLARGO"].str.contains(".+ TASA ")]

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,U##NOMBRE,NOMBRE,NOMLARGO,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,...,TELEFONO1,TELEFONO2,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN
143,NaN,NaN,FISO 80672 CHIH FDO RVA F,FISO 80672 CHIH FDO RVA F,FONDO RESERVA TASA FIJA NACIONAL FINANCIERA F...,1064750,VIGENTE,NaN,NaN,NaN,...,,(614)4293312 Y 4293313,,FEC151125BQ8,NaN,NaN,NaN,Moral Nacional Gravable,NaN,TAS
144,NaN,NaN,FISO 80672 CHIH EMI T FIJA,FISO 80672 CHIH EMI T FIJA,CUENTA EMISIONES TASA FIJA NAFIN FIDEICOMISO ...,1064749,VIGENTE,NaN,NaN,NaN,...,,(614)4193312 Y 4293313,,FEC151125BQ8,NaN,NaN,NaN,Moral Nacional Gravable,NaN,TAS
148,NaN,NaN,FISO80672 CHIH FDO RVA T V,FISO80672 CHIH FDO RVA T V,FONDO RESERVA TASA VARIABLE NACIONAL FINANCIER...,1064752,VIGENTE,NaN,NaN,NaN,...,429 3313,614 429 3312,,FEC151125BQ8,NaN,NaN,NaN,Moral Nacional Gravable,NaN,TAS
149,NaN,NaN,FISO80672 CHIHCTA TASA VAR,FISO80672 CHIHCTA TASA VAR,CUENTA EMISIONES TASA VAR NACIONAL FIANCIERA ...,1064751,VIGENTE,NaN,NaN,NaN,...,,53256048,,FEC151125BQ8,NaN,NaN,NaN,Moral Nacional Gravable,NaN,TAS
540,NaN,NaN,TEMPLETON DEUDA TASA REAL,TEMPLETON DEUDA TASA REAL,FRANKLIN TEMPLETON ASSET MANAGEMENT MEXICO TEM...,1065079,VIGENTE,NaN,NaN,NaN,...,,50020676,,FTD150831QD9,NaN,NaN,NaN,Moral Nacional no Gravable,NaN,TAS
596,NaN,NaN,FDO +TASAA1,Fdo +TASAA1,".+ TASA ACTIVO, S.A. DE C.V., F.I.R.V.",1064707,VIGENTE,NaN,NaN,NaN,...,,55 5033 3341,,STA130219T22,NaN,NaN,NaN,Moral Nacional no Gravable,NaN,TAS
1267,NaN,NaN,VALMX17,VALMX17,FONDO VALMEX 17 EN INSTRUMENTOS DE DEUDA EN TA...,1064158,VIGENTE,NaN,NaN,NaN,...,,52791200,,FVD031015CH7,NaN,NaN,NaN,Moral Nacional no Gravable,NaN,TAS
1268,NaN,NaN,VALMX15,VALMX15,"FONDO VALMEX 15 EN BONOS DE TASA FIJA, S.A. DE...",1064157,VIGENTE,NaN,NaN,NaN,...,,52791200,,FVQ031015G42,NaN,NaN,NaN,Moral Nacional no Gravable,NaN,TAS


In [69]:
df_tas_tran = pl.read_csv("/home/ale1726/proyects/datalake/clientes/data/test/data/productos/TAS/data_origen/21_05_2025/productos_clientes_TAS.dat")

# fideicomisos

In [ ]:
#fideicomisos = df_tas[(df_tas["RFC"].str.contains("NFI3406305T0"))]

In [137]:
fideicomisos = df_tas[df_tas["PROFESION"] == 'FIDEICOMISOS']

In [152]:
fideicomisos["numero_fiso"] = fideicomisos["NOMBRE"].str.extract(r"^(\d{1,5})")

/tmp/ipykernel_2502/3654158293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fideicomisos["numero_fiso"] = fideicomisos["NOMBRE"].str.extract(r"^(\d{1,5})")


In [ ]:
fideicomisos

In [153]:
fideicomisos.value_counts("numero_fiso")

numero_fiso
80139    209
80280     20
80758     11
80773      8
9738       6
80627      5
80745      4
80728      4
80776      3
80732      3
80722      3
80714      3
80686      3
80681      3
80748      2
80664      2
80724      2
11480      2
80581      2
80460      2
80736      1
80841      1
80784      1
2935       1
80774      1
30139      1
80760      1
80005      1
80752      1
80098      1
80105      1
80741      1
80739      1
80734      1
80525      1
80733      1
80370      1
80730      1
80449      1
80469      1
80715      1
80479      1
11701      1
80703      1
80702      1
80698      1
80710      1
Name: count, dtype: int64

In [138]:
fideicomiso_80139 = fideicomisos[
    (fideicomisos["NOMLARGO"].str.contains("80139", case=False, na=False)) | 
    (fideicomisos["NOMBRE"].str.contains("80139", case=False, na=False))
]

fideicomiso_80776 = fideicomisos[
    (fideicomisos["NOMLARGO"].str.contains("80776", case=False, na=False)) | 
    (fideicomisos["NOMBRE"].str.contains("80776", case=False, na=False))
]


fideicomiso_80280 = fideicomisos[
    (fideicomisos["NOMLARGO"].str.contains("80280", case=False, na=False)) | 
    (fideicomisos["NOMBRE"].str.contains("80280", case=False, na=False))
]

In [139]:
fideicomiso_complemento = fideicomisos[
    ~(
        (fideicomisos["NOMLARGO"].str.contains("80139", case=False, na=False)) | 
        (fideicomisos["NOMBRE"].str.contains("80139", case=False, na=False))
    )
]


fideicomiso_complemento = fideicomiso_complemento[
    ~(
        (fideicomiso_complemento["NOMLARGO"].str.contains("80776", case=False, na=False)) | 
        (fideicomiso_complemento["NOMBRE"].str.contains("80776", case=False, na=False))
    )
]

fideicomiso_complemento = fideicomiso_complemento[
    ~(
        (fideicomiso_complemento["NOMLARGO"].str.contains("80280", case=False, na=False)) | 
        (fideicomiso_complemento["NOMBRE"].str.contains("80280", case=False, na=False))
    )
]

In [140]:
fideicomiso_complemento = fideicomiso_complemento.sort_values(by="NOMBRE")

In [143]:
df_filtrado = fideicomiso_complemento[fideicomiso_complemento["NOMBRE"].str.contains(r"^\d", na=False)]

In [145]:
df_filtrado = df_filtrado.sort_values(by="NOMBRE")

In [147]:
df_filtrado["numero_fiso"] = df_filtrado["NOMBRE"].str.extract(r"^(\d{1,5})")

In [148]:
df_filtrado = df_filtrado.sort_values(by="NOMBRE")

In [154]:
df_filtrado.value_counts("numero_fiso")

numero_fiso
80758    11
80773     8
9738      6
80627     5
80745     4
80728     4
80732     3
80722     3
80686     3
80681     3
80714     3
11480     2
80581     2
80664     2
80460     2
80724     2
80748     2
80752     1
80739     1
80741     1
80005     1
80760     1
30139     1
80734     1
2935      1
80774     1
80784     1
80841     1
80736     1
80479     1
80733     1
80098     1
80525     1
80105     1
80370     1
80715     1
11701     1
80710     1
80703     1
80702     1
80698     1
80449     1
80469     1
80730     1
Name: count, dtype: int64

# personas morales

In [102]:
personas_morales = df_tas[df_tas["PROFESION"] != 'FIDEICOMISOS']

In [35]:
personas_morales = personas_morales[personas_morales["RFC"] != 'NFI3406305T0']

In [103]:
personas_morales_filtred = personas_morales[personas_morales["RFC"] != 'XXXXXXXXXXXXX']
personas_morales_complemento = personas_morales[personas_morales["RFC"] == 'XXXXXXXXXXXXX']

In [ ]:
filtred_ctl_rep_mor = (
    personas_morales_filtred
    .groupby("RFC", as_index=False)
    .agg({
        col: (lambda x: ', '.join(map(str, sorted(set(x.dropna()))))) 
        for col in personas_morales_filtred.columns if col != "RFC"
    })
)

filtred_clt_rep_reordenado = filtred_ctl_rep_mor[personas_morales_complemento.columns]


data_pm =  pd.concat([personas_morales_complemento, filtred_ctl_rep_mor], ignore_index=True)


In [ ]:
data_pm